In [1]:
## input final normalized and adjusted file and Mitocarta data + uniprot data to extract all proteins belongs to Mitochondria

In [19]:
## All necessary libraries 

In [20]:
import os
import pandas as pd
import requests
import time
import requests
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt


In [ ]:

print("Current Working Directory:", os.getcwd())



In [ ]:

# file path 
final_norm_path = "./Final_Normalized.csv"

# Load dataset
data = pd.read_csv(final_norm_path)

# Print success message and show the first few rows
print(" File loaded successfully!")
print(data.head(3))

In [ ]:

# file path 
final_mito_path = "./Mitocarta_3.csv"

# Load dataset
data = pd.read_csv(final_mito_path)

# Print success message and show the first few rows
print(" File loaded successfully!")
print(data.head(3))

In [ ]:

# Load the datasets
mitocarta_path ="./Mitocarta_3.csv"
data_path = "./Final_Normalized.csv"

mitocarta_df = pd.read_csv(mitocarta_path)
data_df = pd.read_csv(data_path)

# Step 1: Merge the data based on the 'Accession' column
merged_df = pd.merge(data_df, mitocarta_df[['Accession', 'MitoCarta3.0_SubMitoLocalization']], 
                     on='Accession', how='left')

# Step 2: Add a column to indicate if the 'Accession' is part of the mitochondrial gene data
merged_df['Mitochondrial'] = merged_df['MitoCarta3.0_SubMitoLocalization'].notnull()

folder_path = "./"

# Specify the full file path for the output CSV file
output_path = "./data_mito_merge.csv"

# Save the filtered DataFrame to the specified path
merged_df.to_csv(output_path, index=False)

print(f"merged data saved to: {output_path}")





In [ ]:

# Load the uploaded CSV file
file_path = "./data_mito_merge.csv"
data = pd.read_csv(file_path)


# Filter the rows where the 'Mitochondrial' column is False
not_in_Mitocarta = data[data['Mitochondrial'] == False]

# Save the filtered data to a new CSV file
output_file_path = "./not_in_Mitocarta.csv"
not_in_Mitocarta.to_csv(output_file_path, index=False)

output_file_path


In [ ]:


# Load the not_in_Mitocarta file
file_path = "./not_in_Mitocarta.csv"
data = pd.read_csv(file_path)

# Function to query UniProt and retrieve subcellular localization
def get_uniprot_localization(accession):
    try:
        url = f"https://rest.uniprot.org/uniprotkb/{accession}.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            localization = []
            for comment in data.get("comments", []):
                if comment["commentType"] == "SUBCELLULAR LOCATION":
                    for loc in comment.get("subcellularLocations", []):
                        localization.append(loc["location"]["value"])
            return "; ".join(localization) if localization else "No data found"
        else:
            return "Not found"
    except Exception as e:
        return f"Error: {str(e)}"

# Add a new column for subcellular localization
data["Subcellular_Localization"] = data["Accession"].apply(get_uniprot_localization)

# Save the updated data to a new CSV file
output_file_path = "./not_in_Mitocarta_with_localization.csv"
data.to_csv(output_file_path, index=False)

print(f"Updated file saved as {output_file_path}")


In [27]:
## double check for list of non_Mito

In [28]:
### after this step all data was double check by study and manually correct file "Not_in Mitocarta_correct.csv"

In [ ]:

# Reload the uploaded files
normalized_data = pd.read_csv("./Final_Normalized.csv")
non_mito_data = pd.read_csv("./non_mito_normal_data_corrected.csv")

# Merge the two datasets on 'Accession'
merged_data = pd.merge(non_mito_data, normalized_data, on='Accession', how='left')


# Check the number of unique Accession values in both datasets
normalized_accessions = set(normalized_data['Accession'])
non_mito_accessions = set(non_mito_data['Accession'])


# Save the merged dataset as a new CSV file
output_file_path = "./total_non_mito_final.csv"
merged_data.to_csv(output_file_path, index=False)

# Provide the download link for the new CSV file
output_file_path




In [ ]:
# Load the new dataset and the Final_Normalized dataset
total_mito_data = pd.read_csv("./total_Mito_unip_carta_Subcellular.csv")
final_normalized_data = pd.read_csv("./Final_Normalized.csv")

# Merge the datasets on 'Accession'
merged_total_mito = pd.merge(total_mito_data, final_normalized_data, on='Accession', how='left')

# Save the merged dataset as a new CSV file
output_file_path = "./total_mito_final.csv"
merged_total_mito.to_csv(output_file_path, index=False)

# Provide the download link for the new merged file
output_file_path


In [ ]:
####pie chart for Mito and non_mito

In [ ]:
import matplotlib.pyplot as plt

# Data from the flowchart
mito_proteins = 647
non_mito_proteins = 115
total_proteins = mito_proteins + non_mito_proteins

# Calculate percentages
mito_percentage = (mito_proteins / total_proteins) * 100
non_mito_percentage = (non_mito_proteins / total_proteins) * 100

# Create labels and data for the pie chart
labels = [f'Mitochondrial ({mito_percentage:.1f}%)', f'Non-Mitochondrial ({non_mito_percentage:.1f}%)']
sizes = [mito_percentage, non_mito_percentage]
colors = ['#66b3ff', '#ff9999']

# Create the pie chart
plt.figure(figsize=(5,5))
plt.pie(
    sizes, 
    labels=labels, 
    colors=colors, 
    startangle=90, 
    autopct='%1.1f%%', 
    wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'},
    textprops={'fontsize': 12, 'fontweight': 'bold'}
)

# Draw a white circle in the center to make it a donut chart
centre_circle = plt.Circle((0,0),0.30,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Title
plt.title('Mitochondrial vs Non-Mitochondrial Proteins', fontsize=12, fontweight='bold')

# Save as PDF
piechart_filename = "./mito_nonmito_piechart.pdf"
plt.savefig(piechart_filename, format="pdf", bbox_inches="tight")

# Show the pie chart
plt.show()

print(f"Pie chart saved as {piechart_filename}")


In [ ]:
# Load the new dataset
file_path_non_mito = "./non_mito_normal_data_corrected_2.csv"
data_non_mito = pd.read_csv(file_path_non_mito)

# Group by the 'Corrected_subcellular' column and count occurrences
localization_counts_non_mito = data_non_mito['Corrected_subcellular'].value_counts()

# Take the total count as  (given by the user)
total_count = 765

# Calculate percentages
localization_percentages_non_mito = (localization_counts_non_mito / total_count) * 100

# Sort the percentages in descending order
localization_percentages_non_mito = localization_percentages_non_mito.sort_values(ascending=False)

# Create a horizontal bar plot
import matplotlib.pyplot as plt

plt.figure(figsize=(7,6))
plt.barh(localization_percentages_non_mito.index, localization_percentages_non_mito, color="#33cbff")

# Add percentages at the end of the bars
for index, value in enumerate(localization_percentages_non_mito):
    plt.text(value, index, f'{value:.2f}%', va='center', fontsize=10, fontweight='bold')

# Set labels and title with bold font size 12
plt.xlabel('Percentage (%)', fontsize=12, fontweight='bold')
plt.ylabel('Non-Mitochondrial SubLocalization', fontsize=12, fontweight='bold')
plt.title('Non-Mitochondrial SubLocalization Distribution', fontsize=12, fontweight='bold')

# Remove the spines (borders) around the plot
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Set tick label font sizes to bold 12
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Add black X and Y axis lines with thickness of 1
plt.gca().spines['bottom'].set_linewidth(1)
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_linewidth(1)
plt.gca().spines['left'].set_color('black')

# Show the plot
plt.tight_layout()


# Save the plot as a PDF
pdf_path = "./non_mito_sublocalization_distribution.pdf"
plt.savefig(pdf_path, format="pdf", bbox_inches="tight")

plt.show()


In [ ]:
# Load the new dataset
file_path_mitocarta = "./total_Mito_unip_carta_Subcellular_2.csv"
data_mitocarta = pd.read_csv(file_path_mitocarta)

# Group by the 'sub_mito' column and count occurrences
localization_counts_mitocarta = data_mitocarta['sub_mito'].value_counts()

# Take the total count as 647 (given by the user)
total_mitocarta_count = 647

# Calculate percentages
localization_percentages_mitocarta = (localization_counts_mitocarta / total_mitocarta_count) * 100


# Recreate the optimized bar plot with further refinements

plt.figure(figsize=(6, 4))

# Use a more distinct color with edge outlining
plt.barh(localization_percentages_mitocarta.index, localization_percentages_mitocarta, color='#ff4500', edgecolor='black', alpha=0.85)

# Add percentages at the end of bars with improved alignment
for index, value in enumerate(localization_percentages_mitocarta):
    plt.text(value + 0.5, index, f'{value:.1f}%', va='center', fontsize=11, fontweight='bold', color='black')

# Set labels and title with clear, bold fonts
plt.xlabel('Percentage (%)', fontsize=13, fontweight='bold', color='black')
plt.ylabel('Sub-Mito Localization', fontsize=13, fontweight='bold', color='black')
plt.title('Sub-Mito Localization in MitoCarta 3.0', fontsize=14, fontweight='bold', color='black')

# Enhance visual clarity by removing top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Strengthen bottom and left axis lines for emphasis
plt.gca().spines['bottom'].set_linewidth(1.2)
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_linewidth(1.2)
plt.gca().spines['left'].set_color('black')

# Adjust tick labels for better readability
plt.xticks(fontsize=12, fontweight='bold', color='black')
plt.yticks(fontsize=12, fontweight='bold', color='black')

# Optimize layout for spacing
plt.tight_layout()

# Save the refined plot as a PDF
pdf_path_mitocarta_refined = "./refined_mitocarta_distribution.pdf"
plt.savefig(pdf_path_mitocarta_refined, format="pdf", bbox_inches="tight")

# Show the refined plot
plt.show()



In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
file_path = "./IN_OUT_possible.csv"
df = pd.read_csv(file_path)


# Rename 'Gene_Name' to uppercase for protein names
df['Protein_Name'] = df['Gene_Name'].str.upper()

# Define the groups
groups = {
    'WtSham': ['WtSham_2', 'WtSham_3', 'WtSham_4'],
    'WtLPS': ['WtLPS_1', 'WtLPS_2', 'WtLPS_3', 'WtLPS_4'],
    'WtShamTB': ['WtShamTB_1', 'WtShamTB_2', 'WtShamTB_3'],
    'WtLPSTB': ['WtLPSTB_1', 'WtLPSTB_2', 'WtLPSTB_3', 'WtLPSTB_4'],
    'BtgSham': ['BtgSham_1', 'BtgSham_2', 'BtgSham_3'],
    'BtgLPS': ['BtgLPS_1', 'BtgLPS_2', 'BtgLPS_3', 'BtgLPS_4'],
    'BKSham': ['BKSham_1', 'BKSham_2', 'BKSham_3'],
    'BKLPS': ['BKLPS_1', 'BKLPS_2', 'BKLPS_3']
}

# Create a new DataFrame for group averages with binary values (1 if available, 0 if not)
group_averages = pd.DataFrame({'Protein_Name': df['Protein_Name']})

# Compute binary presence (1 for available values, 0 for NaN)
for group_name, samples in groups.items():
    group_averages[group_name] = df[samples].notna().astype(int).mean(axis=1)  # Average presence

# Set 'Protein_Name' as index
group_averages.set_index('Protein_Name', inplace=True)

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(group_averages, cmap="coolwarm", linewidths=0.5, cbar=True)

# Add title and labels
plt.title("Group-wise Protein Presence Heatmap", fontsize=14, fontweight="bold")
plt.xlabel("Groups", fontsize=12)
plt.ylabel("Protein Name", fontsize=12)

# Save the heatmap as a PDF
heatmap_path = "./group_average_heatmap.pdf"
plt.savefig(heatmap_path, format="pdf", bbox_inches="tight")
plt.show()

